In [3]:
import pandas as pd
import numpy as np
from hmmlearn.hmm import GaussianHMM
import yfinance as yf
import datetime

In [6]:
#Get VIX Data
ticker_symbol = '^VIX'

start_date = '2020-01-01'
end_date = datetime.datetime.now().strftime('%Y-%m-%d')

vix_data = yf.download(ticker_symbol, start=start_date, end=end_date, interval='1d')
print(vix_data.head())

vix_data.to_csv(f'VIX_OHLC_{start_date}_to_{end_date}.csv')


[*********************100%***********************]  1 of 1 completed

1 Failed download:
['^VIX']: YFRateLimitError('Too Many Requests. Rate limited. Try after a while.')


Empty DataFrame
Columns: [(Adj Close, ^VIX), (Close, ^VIX), (High, ^VIX), (Low, ^VIX), (Open, ^VIX), (Volume, ^VIX)]
Index: []


In [2]:
# Load the already merged minute-level data
df = pd.read_csv("data/raw/XAUUSD/combined_data.csv", parse_dates=['timestamp'], index_col='timestamp')
if 'ATR' in df.columns:
    df.drop(columns='ATR', inplace=True)

# Resample to daily OHLC
daily_ohlc = df.resample('1D').agg({
    'high': 'max',
    'low': 'min',
    'close': 'last'
})

# Calculate previous day's close
daily_ohlc['prev_close'] = daily_ohlc['close'].shift(1)

# Calculate True Range (TR)
daily_ohlc['true_range'] = daily_ohlc.apply(
    lambda row: max(
        row['high'] - row['low'],
        abs(row['high'] - row['prev_close']),
        abs(row['low'] - row['prev_close'])
    ) if pd.notnull(row['prev_close']) else None,
    axis=1
)

#Calculate 14-day ATR (excluding current day via shift)
daily_ohlc['atr_14'] = daily_ohlc['true_range'].rolling(window=14).mean().shift(1)

#Calculate moving averages on daily close (also shifted)
daily_ohlc['ma_14'] = daily_ohlc['close'].rolling(window=14).mean().shift(1)
daily_ohlc['ma_30'] = daily_ohlc['close'].rolling(window=30).mean().shift(1)
daily_ohlc['ma_100'] = daily_ohlc['close'].rolling(window=100).mean().shift(1)

# Rolling max/min of the daily close
daily_ohlc['max_14'] = daily_ohlc['close'].rolling(window=14).max().shift(1)
daily_ohlc['min_14'] = daily_ohlc['close'].rolling(window=14).min().shift(1)

daily_ohlc['max_30'] = daily_ohlc['close'].rolling(window=30).max().shift(1)
daily_ohlc['min_30'] = daily_ohlc['close'].rolling(window=30).min().shift(1)

daily_ohlc['max_100'] = daily_ohlc['close'].rolling(window=100).max().shift(1)
daily_ohlc['min_100'] = daily_ohlc['close'].rolling(window=100).min().shift(1)

# Cumulative max of close price
daily_ohlc['static_peak'] = daily_ohlc['close'].cummax()

# Static drawdown from that peak
daily_ohlc['drawdown_static'] = (
    (daily_ohlc['close'] - daily_ohlc['static_peak']) / daily_ohlc['static_peak']
).shift(1)

daily_ohlc['drawdown_30'] = (
    (daily_ohlc['close'] - daily_ohlc['close'].rolling(window=30).max()) 
    / daily_ohlc['close'].rolling(window=30).max()
).shift(1)

# Forward-fill daily values into minute-level data
df['daily_high'] = daily_ohlc['high'].reindex(df.index, method='ffill')
df['daily_low'] = daily_ohlc['low'].reindex(df.index, method='ffill')
df['daily_close'] = daily_ohlc['close'].reindex(df.index, method='ffill')
df['true_range'] = daily_ohlc['true_range'].reindex(df.index, method='ffill')
df['atr_14'] = daily_ohlc['atr_14'].reindex(df.index, method='ffill')
df['ma_14'] = daily_ohlc['ma_14'].reindex(df.index, method='ffill')
df['ma_30'] = daily_ohlc['ma_30'].reindex(df.index, method='ffill')
df['ma_100'] = daily_ohlc['ma_100'].reindex(df.index, method='ffill')

#Day and Weeknumber
df['day_of_week'] = df.index.day_name().str[:3]
df['week_number'] = df.index.isocalendar().week

df['max_price_14'] = daily_ohlc['max_14'].reindex(df.index, method='ffill')
df['min_price_14'] = daily_ohlc['min_14'].reindex(df.index, method='ffill')

df['max_price_30'] = daily_ohlc['max_30'].reindex(df.index, method='ffill')
df['min_price_30'] = daily_ohlc['min_30'].reindex(df.index, method='ffill')

df['max_price_100'] = daily_ohlc['max_100'].reindex(df.index, method='ffill')
df['min_price_100'] = daily_ohlc['min_100'].reindex(df.index, method='ffill')

#Drawdown
df['drawdown_static'] = daily_ohlc['drawdown_static'].reindex(df.index, method='ffill')
df['drawdown_30'] = daily_ohlc['drawdown_30'].reindex(df.index, method='ffill')

import numpy as np
from hmmlearn.hmm import GaussianHMM

# 1. Compute HMM Features on Daily Data
daily_ohlc['log_return'] = np.log(daily_ohlc['close'] / daily_ohlc['close'].shift(1))
daily_ohlc['volatility'] = (
    daily_ohlc['close']
    .rolling(window=10)
    .apply(lambda x: np.mean((x - x.mean())**2))
).shift(1)

# 2. Drop NaNs (due to rolling/shift)
hmm_features = daily_ohlc[['log_return', 'volatility']].dropna()

# 3. Fit HMM on ALL data (no slicing)
model = GaussianHMM(n_components=3, covariance_type='full', n_iter=75, random_state=42)
model.fit(hmm_features)
hmm_features['regime'] = model.predict(hmm_features)

# 4. Label Regimes (Bull, Bear, Neutral) based on return means
regime_means = hmm_features.groupby('regime')['log_return'].mean().sort_values()
regime_mapping = {
    regime_means.index[0]: 'Bear',
    regime_means.index[1]: 'Neutral',
    regime_means.index[2]: 'Bull'
}
hmm_features['regime_label'] = hmm_features['regime'].map(regime_mapping)

# 5. Merge regime info into daily_ohlc
daily_ohlc['regime'] = hmm_features['regime']
daily_ohlc['regime_label'] = hmm_features['regime_label']

# 6. Forward-fill daily regime into minute-level df
df['regime'] = daily_ohlc['regime'].reindex(df.index, method='ffill')
df['regime_label'] = daily_ohlc['regime_label'].reindex(df.index, method='ffill')


#Optional: Inspect regime distribution
print(df['regime_label'].value_counts())

# Overwrite the original file
df.to_csv("data/processed/XAUUSD/combined_data.csv")

Neutral    1114560
Bull        839520
Bear        593280
Name: regime_label, dtype: int64
